# Preperations

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#! pip install transformers
#! pip install accelerate
! pip install -U bitsandbytes
#! pip install loralib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 9.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import cv2
from PIL import Image
import requests
import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration
from transformers import BitsAndBytesConfig

Folders structure:

In [ ]:
%cd '/content/drive/My Drive/Colab Notebooks/Deep learning/LLaVa_food_grade_project'
%ls
import os
path = os.getcwd()
print('path: ' + path)
OutputPath = path + '/Output'
ModelPath = path + '/Models'
DataPath = path + '/Data'

/content/drive/My Drive/Colab Notebooks/Deep learning/LLaVa_food_grade_project
 Chat_GPT_rating_accuracy.ipynb   GPT2_prediction.ipynb   Output/
 CNN_rating_prediction.ipynb      LLaVa_Inference.ipynb  'עותק של LLaVa_Inference.ipynb'
 Data/                            Models/
path: /content/drive/My Drive/Colab Notebooks/Deep learning/LLaVa_food_grade_project


load the reviews:

In [ ]:
df = pd.read_csv(DataPath+'/reviews.csv')

print(df.head(5).to_string())

                     business_name    author_name                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                text                                   

make a df of only the taste reviews:

In [ ]:
df_of_tastes = df[df['rating_category'] == 'taste'].copy()
df_of_tastes.reset_index(drop=True, inplace=True)
print(df_of_tastes.head(5).to_string())

                     business_name   author_name                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      text                                        photo  rating rating_category
0  Haci'nin Yeri - Yigit Lokantasi   Gulsum Akar  We went to Marmaris with my wife for a holiday. We chose this restaurant as a place for dinner based on the reviews and because we wanted juicy food. 

make a df of only the taste ratings:

In [ ]:
df_of_tastes_ratings = df_of_tastes.drop(columns=['business_name','author_name','text','photo','rating_category'])
print(df_of_tastes_ratings.head(5).to_string())

   rating
0       5
1       5
2       5
3       5
4       2


download llava model:

In [ ]:
# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"

# Define quantization configuration
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

# Initialize model
model_id = "llava-hf/llava-1.5-7b-hf"

model = None

# Check if model is already configured and loaded
if model is None:
    processor = AutoProcessor.from_pretrained(model_id)
    model = LlavaForConditionalGeneration.from_pretrained(
        model_id,
        quantization_config=quantization_config,
        device_map="auto"  # Use GPU if available
    )
    model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

# LLavA inference

In [ ]:
# the question for the AI
#prompt = "USER: <image>\nDescribe the dish? ASSISTANT:"
prompt = "USER: <image>\nDescribe the dish in detail, including its ingredients, presentation, portion size, and how appetizing it appears based on the image.\nASSISTANT:"

LLaVa Inference based on images only

In [ ]:
import time

In [ ]:
# Define the batch size
batch_size = 8

# Initialize the output file path
output_file = OutputPath + "/llava_inference_images.csv"

# Check if the output file exists and read the last saved batch's index
if os.path.exists(output_file):
    descriptions = pd.read_csv(output_file)
    last_processed_index = descriptions['index'].max() + 1  # Resume from the next index
else:
    descriptions = pd.DataFrame(columns=['index', 'description'])
    last_processed_index = 0  # Start from the beginning

# Start time tracking
start_time = time.time()

# Loop through the dataframe starting from the last processed index
for start_idx in range(last_processed_index, len(df_of_tastes), batch_size):
    batch_start_time = time.time()  # Start time for the current batch
    batch = df_of_tastes.iloc[start_idx:start_idx+batch_size]

    batch_descriptions = []

    for ind, row in batch.iterrows():
        image_path = DataPath + '/' + row['photo']

        # Read and process the image
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        inputs = processor(text=prompt, images=image, return_tensors="pt")

        # Generate description from the model
        generate_ids = model.generate(**inputs, max_new_tokens=200)
        description = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

        # Remove the prompt part by splitting the description at "ASSISTANT:" and keeping only the response
        description = description.split("ASSISTANT:")[-1].strip()

        # Store the index and description in the temporary batch list
        batch_descriptions.append({'index': ind, 'description': description})

    # Convert batch to DataFrame
    batch_df = pd.DataFrame(batch_descriptions)
    descriptions = pd.concat([descriptions, batch_df], ignore_index=True)
    descriptions.to_csv(output_file, index=False)

    # Log time taken for the current batch
    print(f"Processed {min(start_idx+batch_size, len(df_of_tastes))} out of {len(df_of_tastes)} images in {time.time() - batch_start_time:.2f} seconds")

# Final save in case the loop finishes normally
descriptions.to_csv(output_file, index=False)

# Log total time taken
total_time = time.time() - start_time
print(f"Total processing time: {total_time:.2f} seconds.")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Processed 40 out of 330 images in 75.88 seconds
Processed 48 out of 330 images in 76.35 seconds
Processed 56 out of 330 images in 80.98 seconds
Processed 64 out of 330 images in 82.01 seconds
Processed 72 out of 330 images in 75.64 seconds
Processed 80 out of 330 images in 73.95 seconds
Processed 88 out of 330 images in 74.38 seconds
Processed 96 out of 330 images in 73.53 seconds
Processed 104 out of 330 images in 74.40 seconds
Processed 112 out of 330 images in 77.55 seconds
Processed 120 out of 330 images in 79.56 seconds
Processed 128 out of 330 images in 80.71 seconds
Processed 136 out of 330 images in 72.81 seconds
Processed 144 out of 330 images in 70.53 seconds
Processed 152 out of 330 images in 75.97 seconds
Processed 160 out of 330 images in 71.36 seconds
Processed 168 out of 330 images in 75.91 seconds
Processed 176 out of 330 images in 71.59 seconds
Processed 184 out of 330 images in 78.50 seconds
Processed 192 out of 330 images in 74.24 seconds
Processed 200 out of 330 ima

Generate Inferences based on images and customer review

In [ ]:
# Track time for generating inferences
start_time = time.time()

# Initialize results DataFrame
output_file = OutputPath + '/llava_inference_text_images.csv'

# Check if the output file exists, if not create it
if os.path.exists(output_file):
    inferences = pd.read_csv(output_file)
    last_processed_index = inferences['index'].max() + 1
else:
    inferences = pd.DataFrame(columns=['index', 'inference'])
    last_processed_index = 0

# Loop through the dataframe from the last processed index
for index in range(last_processed_index, len(df_of_tastes)):
    row = df_of_tastes.iloc[index]
    image_path = DataPath + '/' + row['photo']
    review_text = row['text']

    # Process image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Prepare prompt for inference
    prompt = f"USER: <image>\nHere is the review: {review_text}\nProvide a detailed description of this dish including presentation, ingredients, and overall appeal. ASSISTANT:"

    # Prepare inputs
    inputs = processor(text=prompt, images=image, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Generate inference
    generate_ids = model.generate(**inputs, max_new_tokens=200)
    inference_raw = processor.batch_decode(generate_ids, skip_special_tokens=True)[0]

    # Split the response to get only the part after "ASSISTANT:"
    if "ASSISTANT:" in inference_raw:
        inference_cleaned = inference_raw.split("ASSISTANT:")[-1].strip()
    else:
        inference_cleaned = inference_raw.strip()

    # Save inference
    new_row = pd.DataFrame({'index': [index], 'inference': [inference_cleaned]})
    inferences = pd.concat([inferences, new_row], ignore_index=True)

    # Save the progress after each iteration
    inferences.to_csv(output_file, index=False)

# Log time taken
print(f"Inferences generation completed in {time.time() - start_time} seconds.")


Inferences generation completed in 2742.280478954315 seconds.


# LLaVa predictions

Produce a Rating Based on Images and Review Text:

In [ ]:

import re
from sklearn.metrics import accuracy_score

# Track time for rating based on images and text
start_time = time.time()

# Initialize results DataFrame
output_file = OutputPath + '/combined_ratings_LLAVA_img_n_text.csv'

# Check if the output file exists and load the data
if os.path.exists(output_file):
    combined_ratings = pd.read_csv(output_file)
    last_processed_index = combined_ratings['index'].max() + 1
else:
    combined_ratings = pd.DataFrame(columns=['index', 'predicted_rating', 'actual_rating'])
    last_processed_index = 0

rating_regex = re.compile(r'rate (?:the|this) dish a (\d) out of 5')

# Store actual and predicted ratings for accuracy calculation
all_predicted_ratings = combined_ratings['predicted_rating'].tolist()
all_actual_ratings = combined_ratings['actual_rating'].tolist()

# Check if there are unprocessed rows
if last_processed_index >= len(df_of_tastes):
    print("All rows have been processed. No unprocessed data found.")
else:
    # Loop through the dataframe from the last processed index
    for index in range(last_processed_index, len(df_of_tastes)):
        row = df_of_tastes.iloc[index]
        image_path = DataPath + '/' + row['photo']
        review_text = row['text']
        actual_rating = row['rating']  # Actual rating from the dataframe

        # Process image
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Prompt
        prompt = f"USER: <image>\nHere is the review: {review_text}\nBased on both the image and the review, how would you rate this dish on a scale of 1 to 5? ASSISTANT:"

        # Prepare inputs
        inputs = processor(text=prompt, images=image, return_tensors="pt")

        # Generate prediction
        generate_ids = model.generate(**inputs, max_new_tokens=50)
        prediction_raw = processor.batch_decode(generate_ids, skip_special_tokens=True)[0]

        # Extract the rating
        match = rating_regex.search(prediction_raw)
        if match:
            predicted_rating = int(match.group(1))
        else:
            print(f"Warning: Non-numeric prediction '{prediction_raw}' for index {index}")
            predicted_rating = -1  # Default invalid value

        # Append the actual and predicted ratings
        all_predicted_ratings.append(predicted_rating)
        all_actual_ratings.append(actual_rating)

        # Save predicted rating and actual rating to the DataFrame
        new_row = pd.DataFrame({
            'index': [index],
            'predicted_rating': [predicted_rating],
            'actual_rating': [actual_rating]
        })
        combined_ratings = pd.concat([combined_ratings, new_row], ignore_index=True)

        # Save the progress after each iteration
        combined_ratings.to_csv(output_file, index=False)

# Filter out invalid predictions (-1)
valid_ratings_mask = [p != -1 for p in all_predicted_ratings]  # Filter out invalid ratings (-1)
filtered_predicted_ratings = [p for i, p in enumerate(all_predicted_ratings) if valid_ratings_mask[i]]
filtered_actual_ratings = [a for i, a in enumerate(all_actual_ratings) if valid_ratings_mask[i]]

# Count valid rows
valid_row_count = len(filtered_predicted_ratings)

if valid_row_count > 0:
    # Calculate accuracy if there are valid predictions
    accuracy = accuracy_score(filtered_actual_ratings, filtered_predicted_ratings)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Number of valid rows used for accuracy calculation: {valid_row_count}")
else:
    print("No valid predictions to calculate accuracy.")

# Log time taken
print(f"Combined image and text-based ratings completed in {time.time() - start_time} seconds.")


All rows have been processed. No unprocessed data found.
Accuracy: 32.11%
Number of valid rows used for accuracy calculation: 327
Combined image and text-based ratings completed in 0.010778427124023438 seconds.


LLaVa prediction Based on Food Images

In [ ]:
from sklearn.metrics import accuracy_score
import os
import pandas as pd
import time
import torch
import cv2

start_time = time.time()

# Initialize results DataFrame
output_file = OutputPath + '/image_based_ratings.csv'

# Check if the output file exists
if os.path.exists(output_file):
    image_based_ratings = pd.read_csv(output_file)
    last_processed_index = image_based_ratings['index'].max() + 1
else:
    image_based_ratings = pd.DataFrame(columns=['index', 'predicted_rating', 'actual_rating'])
    last_processed_index = 0

# List to store true ratings
true_ratings = []

# Loop through the dataframe from the last processed index
for idx in range(last_processed_index, len(df_of_tastes)):
    row = df_of_tastes.iloc[idx]
    image_path = DataPath + '/' + row['photo']

    # Process image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Prompt
    prompt = ("USER: <image>\nPlease carefully evaluate the food shown in this image and "
              "rate its quality on a scale of 1 to 5, where 1 is poor and 5 is excellent. "
              "Consider the presentation, freshness, and how appetizing it looks. Respond "
              "with just the rating number. ASSISTANT:")

    # Prepare inputs
    inputs = processor(text=prompt, images=image, return_tensors="pt")

    # Ensure inputs are on the same device as the model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Generate prediction with temperature and top_k for diversity
    generate_ids = model.generate(**inputs, max_new_tokens=50, temperature=0.7, top_k=50, do_sample=True)
    prediction_raw = processor.batch_decode(generate_ids, skip_special_tokens=True)[0]

    # Extract the rating from the raw prediction
    if "ASSISTANT:" in prediction_raw:
        prediction_cleaned = prediction_raw.split("ASSISTANT:")[-1].strip()
    else:
        prediction_cleaned = prediction_raw.strip()

    # Ensure that prediction is numeric (if necessary, convert it)
    try:
        # Try to extract a numeric rating from the cleaned response
        prediction = int(prediction_cleaned)
    except ValueError:
        print(f"Warning: Non-numeric prediction '{prediction_cleaned}' for index {idx}")
        prediction = -1  # Default invalids value

    # Append the true rating (if available)
    true_rating = row['rating']
    true_ratings.append(true_rating)

    # Append the new row to the DataFrame
    new_row = pd.DataFrame({'index': [idx], 'predicted_rating': [prediction], 'actual_rating': [true_rating]})
    image_based_ratings = pd.concat([image_based_ratings, new_row], ignore_index=True)

    # Save the progress after each iteration
    image_based_ratings.to_csv(output_file, index=False)

# Calculate accuracy
predicted_ratings = image_based_ratings['predicted_rating'].tolist()
accuracy = accuracy_score(true_ratings[:len(predicted_ratings)], predicted_ratings)

# Log time taken and accuracy
print(f"Image-based ratings completed in {time.time() - start_time} seconds.")
print(f"Accuracy of predictions: {accuracy * 100:.2f}%")


The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


Image-based ratings completed in 767.827231168747 seconds.
Accuracy of predictions: 23.33%


In [ ]:
len(true_ratings)

330